In [ ]:
import ipychart as ipyc

In [ ]:
chart = ipyc.BarChart()
chart.set_data([10, 20, 30, 40, 50, 80])
chart

In [1]:
import ipywidgets as widgets
from traitlets import Unicode, default, List

class DevChart(widgets.DOMWidget):
    """An example widget."""
    _view_name = Unicode('DevChartView').tag(sync=True)
    _view_module = Unicode('dev').tag(sync=True)
    
    _data = List([]).tag(sync=True)
    _label = List([]).tag(sync=True)
    
    _type = Unicode().tag(sync=True)
    
    def __init__(self, data, kind, options=None):
        
        super().__init__()
        assert kind in ['line', 'bar', 'radar', 'doughnut', 'polarArea', 'bubble', 'horizontalBar'], 'Type must be one of : line, bar, radar, doughnut, polarArea, bubble'
        assert isinstance(data, (list, dict)), 'Please enter data as dict of list'
        
        if isinstance(data, list):
            self._data = data
            self._label = [str(i+1) for i in range(len(self._data))]
            
        else:
            self._data = data['data'] if 'data' in data else None
            self._label = data['labels'] if 'labels' in data else [str(i+1) for i in range(len(self._data))]
        
        self._type = kind

    @default('layout')
    def _default_layout(self):
        return widgets.Layout(height='auto', align_self='stretch')
    

In [2]:
%%javascript
require.undef('dev');


define('dev', ["@jupyter-widgets/base", "./js/node_modules/chart.js/dist/Chart.min.js"], function(widgets) {
    var DevChartView = widgets.DOMWidgetView.extend({
    render: function() {
        
        var data = this.model.get("_data");
        var label = this.model.get("_label");
        var type = this.model.get("_type");
        
        let canvas = document.createElement("canvas");
        var ctx = canvas.getContext('2d');
        
        new Chart(ctx, {
            type: type,
            data: {
                labels: label,
                datasets: [{
                    label: '# of Votes',
                    data: data,
                    backgroundColor: [
                        'rgba(255, 99, 132, 0.2)',
                        'rgba(54, 162, 235, 0.2)',
                        'rgba(255, 206, 86, 0.2)',
                        'rgba(75, 192, 192, 0.2)',
                        'rgba(153, 102, 255, 0.2)',
                        'rgba(255, 159, 64, 0.2)'
                    ],
                    borderColor: [
                        'rgba(255, 99, 132, 1)',
                        'rgba(54, 162, 235, 1)',
                        'rgba(255, 206, 86, 1)',
                        'rgba(75, 192, 192, 1)',
                        'rgba(153, 102, 255, 1)',
                        'rgba(255, 159, 64, 1)'
                    ],
                    borderWidth: 1
                }]
            },
            options: {
                scales: {
                    yAxes: [{
                            gridLines: {
                              display: true,
                          },
                        ticks: {
                            beginAtZero: true
                        }
                    }],
                    xAxes: [{
                          gridLines: {
                          display: false,
                      },
                    }],
                }
            }
        });
        this.el.appendChild(canvas);
        }
    });
    
    return {
        DevChartView : DevChartView
    };
});

<IPython.core.display.Javascript object>

In [3]:
test_wd = DevChart([12, 19, 3, 5, 2, 3], 'horizontalBar')
test_wd

DevChart(layout=Layout(align_self='stretch', height='auto'))